# Playground for new packages

## HyperOpt
## Catboost
## Sklearn pipelines, pipeline custom transformers
## using seaborn and bokeh for visualization
## catboost for an algo
## use pytest to test .py files
## Use docker, flask, gunicorn to deploy and API
### From there it would be deep learning with Keras and GLOVE for NLP.  You could use isolation forests or seq2seq for anomaly detection

# Importation

In [76]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [77]:
file_pkl = '/Users/charisameeker/Documents/Data/iowa_liquor_clean3.4.cats.pkl'
df = pd.read_pickle(file_pkl)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [78]:
df.county = df.county.cat.add_categories('nan').fillna('nan')
df.store_subname = df.store_subname.cat.add_categories('nan').fillna('nan')

In [79]:
df = df.head(1000)

## We used the below data on Oct 29.

In [108]:
file_pkl = '/Users/charisameeker/Documents/Data/iowa_liquor_clean3.5.cats.1000.pkl'
df.to_pickle(file_pkl)

# Prepare Data

In [81]:
y = df['fd'].copy().values # fd is the result of applying boxcox on the sale column (fitted data)
X = df.copy().drop(['sale', 'fd'], axis=1)
X_columns = X.copy().columns

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# SKLearn Pipeline

In [94]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV


numeric_features = ["year", "month", "day", 'store_number', 'zip_code', 'county_number', \
                    "category", "vendor_number", "item_number", "pack", 'bottle_vol', \
                   'bottle_cost', 'state_bottle_retail', 'bottles_sold',\
                   'vol_sold', 'store_subnumber']
numeric_transformer = [("scaler", StandardScaler())]
numeric_transformer = Pipeline(numeric_transformer)

#pipeline for categorical features
categorical_features = ["store_name", "city", "zip_code", "category", 'category_name',\
                       'vendor_name', 'item_description', 'store_subname']
categorical_transformer = [("encoder", OneHotEncoder(handle_unknown='ignore'))]
categorical_transformer = Pipeline(categorical_transformer)

#divernce of the pipelines
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features), \
                  ("cat", categorical_transformer, categorical_features)])

#incorporating the random forest estimator
pipeline = make_pipeline(preprocessor)

#grid search parameters
params = {"randomforestregressor__n_estimators" : [75, 100, 150]}

pipe = make_pipeline(preprocessor)

# 1 pipeline for data transformation
# save as csv
# for loop with sklearn algos, print/plot
#.   inspect default params and feature importantance
# decide on model
# 1 pipeline with model and gridsearch

In [100]:
type(pipe)

sklearn.pipeline.Pipeline

In [101]:
vd = pipe.fit(df)

In [102]:
vd

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'month', 'day',
                                                   'store_number', 'zip_code',
                                                   'county_number', 'category',
                                                   'vendor_number',
                                                   'item_number', 'pack',
                                                   'bottle_vol', 'bottle_cost',
                                                   'state_bottle_retail',
                                                   'bottles_sold', 'vol_sold',
                                                   'store_subnumber']),
                                                 ('cat',
  

In [103]:
vd.transform(df)

<1000x499 sparse matrix of type '<class 'numpy.float64'>'
	with 21000 stored elements in Compressed Sparse Row format>

In [105]:
df1 = pd.DataFrame(vd.transform(df))

In [72]:
sorted(pipeline.get_params().items())

[('columntransformer',
  ColumnTransformer(transformers=[('num',
                                   Pipeline(steps=[('scaler', StandardScaler())]),
                                   ['year', 'month', 'day', 'store_number',
                                    'zip_code', 'county_number', 'category',
                                    'vendor_number', 'item_number', 'pack',
                                    'bottle_vol', 'bottle_cost',
                                    'state_bottle_retail', 'bottles_sold',
                                    'vol_sold', 'store_subnumber']),
                                  ('cat',
                                   Pipeline(steps=[('encoder', OneHotEncoder())]),
                                   ['city', 'zip_code', 'category',
                                    'category_name', 'vendor_name',
                                    'item_description', 'store_subname'])])),
 ('columntransformer__cat', Pipeline(steps=[('encoder', OneHotEncoder())]))

# HyperOpt

In [86]:
from sklearn.metrics import accuracy_score
from hpsklearn import HyperoptEstimator
from hpsklearn import any_regressor
from hpsklearn import any_preprocessing
from hyperopt import tpe

estim = HyperoptEstimator(classifier=any_regressor('my_clf'),
preprocessing=any_preprocessing('my_pre'),
algo=tpe.suggest,
max_evals=3,
trial_timeout=120)

estim.fit(X_train, y_train)

# Show the results
print(f'Score: {estim.score(X_test, y_test)}')

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

job exception: float() argument must be a string or a number, not 'Timestamp'



  0%|          | 0/1 [00:03<?, ?trial/s, best loss=?]


TypeError: float() argument must be a string or a number, not 'Timestamp'